In [1]:
# Note to import from .py files, must follow structure
# from <.py filename excluding '.py'> import <class name>

# Importing necessary models
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import seaborn as sns
import matplotlib.dates as dates
import matplotlib.ticker as ticker
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import sched, time
from pandas_datareader.data import Options
from py_vollib.black_scholes_merton.implied_volatility import *
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.plotly as py
import plotly
import statsmodels.api as sm
from scipy.stats import skewnorm as skn
from scipy.stats import norm

# Using plotly api_key credentials
# plotly.tools.set_credentials_file(username='aspiringfastlaner', api_key='')

C:\Users\Fang\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning:

The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.



### Pulling all historical data and collapsing into raw dataframe for use
The following cell has code to pull data from yahoo finance for GSPC, and from
CBOE for VIX, VVIX, and SKEW.

#### The final product of the cell is a dataframe named, df, that stores all raw historical yahoo finance data

In [32]:
class datacollect:
        
    # Use six to import urllib so it is working for Python2/3
    from six.moves import urllib
    # If you don't want to use six, please comment out the line above
    # and use the line below instead (for Python3 only).
    #import urllib.request, urllib.parse, urllib.error

    '''
    Starting on May 2017, Yahoo financial has terminated its service on
    the well used EOD data download without warning. This is confirmed
    by Yahoo employee in forum posts.
    Yahoo financial EOD data, however, still works on Yahoo financial pages.
    These download links uses a "crumb" for authentication with a cookie "B".
    This code is provided to obtain such matching cookie and crumb.
    '''

    # Build the cookie handler
    cookier = urllib.request.HTTPCookieProcessor()
    opener = urllib.request.build_opener(cookier)
    urllib.request.install_opener(opener)

    # Cookie and corresponding crumb
    _cookie = None
    _crumb = None

    _headers={'User-Agent': 'Mozilla/5.0 (X11; U; Linux i686) Gecko/20071127 Firefox/2.0.0.11'}

    def get_cookie_crumb():
        '''
        This function perform a query and extract the matching cookie and crumb.
        '''

        # Perform a Yahoo financial lookup on SP500
        req = urllib.request.Request('https://finance.yahoo.com/quote/^GSPC', headers=_headers)
        f = urllib.request.urlopen(req)
        alines = f.read().decode('utf-8')

        # Extract the crumb from the response
        global _crumb
        cs = alines.find('CrumbStore')
        cr = alines.find('crumb', cs + 10)
        cl = alines.find(':', cr + 5)
        q1 = alines.find('"', cl + 1)
        q2 = alines.find('"', q1 + 1)
        crumb = alines[q1 + 1:q2]
        _crumb = crumb

        # Extract the cookie from cookiejar
        global cookier, _cookie
        for c in cookier.cookiejar:
            if c.domain != '.yahoo.com':
                continue
            if c.name != 'B':
                continue
        _cookie = c.value

        # Print the cookie and crumb
        # print('Cookie:', _cookie)
        # print('Crumb:', _crumb)
        return _crumb

    # Downloading directly from yahoo finance spx or vvix data
    def yahoo_historical(ticker = 'SPX'):
        # Using requests to ping yahoo finance to retrieve 
        # historical data table

        # Getting cookie crumb for yahoo finance query
        get_cookie_crumb()

        if ticker == 'VVIX':
            site = 'https://query1.finance.yahoo.com/v7/finance/download/%5EVVIX?period1=1167811200&period2=' + str(int(time.time())) + '&interval=1d&events=history&crumb=' + get_cookie_crumb().replace('\\','')
        else:
            site = 'https://query1.finance.yahoo.com/v7/finance/download/%5EGSPC?period1=-630950400&period2=' + str(int(time.time())) + '&interval=1d&events=history&crumb=' + get_cookie_crumb().replace('\\','')

        df = pd.read_csv(site)
        return df

    # Reading in Data
    # Reading VIX data from CBOE directly
    # VIX is stored as 3 separate files on CBOE's website
    #   2004 to present : http://www.cboe.com/publish/scheduledtask/mktdata/datahouse/vixcurrent.csv
    #   1990 to 2003    : http://www.cboe.com/publish/scheduledtask/mktdata/datahouse/vixarchive.xls
    #   1986 to 2003 VXO: http://www.cboe.com/publish/scheduledtask/mktdata/datahouse/vxoarchive.xls

    # First read raw files directly 
    vix_present = pd.read_csv('http://www.cboe.com/publish/scheduledtask/mktdata/datahouse/vixcurrent.csv').dropna()
    # vix_old = pd.read_excel('http://www.cboe.com/publish/scheduledtask/mktdata/datahouse/vixarchive.xls').dropna()
    vxo_old = pd.read_excel('http://www.cboe.com/publish/scheduledtask/mktdata/datahouse/vxoarchive.xls').dropna()

    # Function for cleaning CBOE VIX data
    def clean_cboe(df):
        df.columns = ['Date','Open','High','Low','Close']
        df = df[1:]
        df['Date'] = pd.to_datetime(df['Date'])
        df = df.set_index(pd.DatetimeIndex(df['Date']))
        return df[['Open','High','Low','Close']]

    # Applying clean_cboe to vix data
    vix_present = clean_cboe(vix_present)
    # vix_old = clean_cboe(vix_old)
    vxo_old = clean_cboe(vxo_old)

    # Currently the vix_old dataframe doesn't have the Open prices so VXO will be used to proxy VIX prior
    # to 2003
    vix = pd.concat([vxo_old,vix_present],axis = 0)

    # Reading SKEW Index data directly from CBOE
    skew = pd.read_csv('https://www.cboe.com/publish/scheduledtask/mktdata/datahouse/skewdailyprices.csv')
    skew_raw = skew.copy()
    skew.columns = ['Date','Skew','na1','na2']
    skew = skew[1:]
    skew['Date'] = pd.to_datetime(skew['Date'])
    skew = skew.set_index(pd.DatetimeIndex(skew['Date']))[['Skew']]
    skew['skew'] = -(pd.to_numeric(skew['Skew'], downcast='float') - 100)/10
    del skew['Skew']

    # Reading in SPX Data
    spx = yahoo_historical()
    spx = spx.set_index(pd.DatetimeIndex(spx['Date']))[['Open','High','Low','Close','Adj Close']]

    # Reading in VVIX Data
    # vvix = yahoo_historical('VVIX')
    # vvix = vvix.set_index(pd.DatetimeIndex(vvix['Date']))[['Open','High','Low','Close','Adj Close']]

    # Joining all index together to one dataframe
    spx = spx[['Open','Close']]
    spx.columns = ['SPX ' + s for s in spx.columns.tolist()]

    vix = vix[['Open','Close']]
    vix.columns = ['VIX ' + s for s in vix.columns.tolist()]

    # vvix = vvix[['Open','Close']]
    # vvix.columns = ['VVIX ' + s for s in vvix.columns.tolist()]

    #
    df = pd.concat([spx,vix,skew],axis = 1).dropna() # Currently excluding VVIX

    # An error in data: 2000-10-18 VIX Close value is a string, converting to float
    df['VIX Close'][2714] = 32.5

    # Fixing VIX values so that they are floats
    df['VIX Close'] = df['VIX Close'].astype('float')
    df['VIX Open'] = df['VIX Open'].astype('float')

    # Adjusting VIX so that it's on 252 trading days
    df['Daily VIX Open'] = np.sqrt(((df['VIX Open']*df['VIX Open'])/365)*1.5)/100
    df['Daily VIX Close'] = np.sqrt(((df['VIX Close']*df['VIX Close'])/365)*1.5)/100

    # Cleaning up unused dataframes
    del skew, spx, vix, vix_present, vxo_old
    

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:95: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



### User-Defined functions

Below cell holds all necessary functions for VaR calculations
- latest_yahoo: Pulls latest yahoo data for SPX, VIX, VVIX, and SKEW

In [10]:
# Pulling Yahoo live data

'''
Function for pulling latest SPX, VIX, VVIX, or SKEW data. Input is a string, pulls 
the latest 2 lines of data from yahoo finance for given ticker and returns a 
dataframe of the open and close with the latest date as the first row.
'''
def latest_yahoo(ticker = 'SPX'):
    # Using requests to ping yahoo finance to retrieve 
    # historical data table
    if ticker == 'VIX':
        site = 'https://finance.yahoo.com/quote/%5EVIX/history?p=^VIX'
    elif ticker == 'VVIX':
        site = 'https://finance.yahoo.com/quote/%5EVVIX/history?p=^VVIX'
    elif ticker == 'SKEW':
        site = 'https://finance.yahoo.com/quote/%5ESKEW/history?p=^SKEW'
    else:
        site = 'https://finance.yahoo.com/quote/%5EGSPC/history?p=^GSPC'
        
    res = requests.get(site)
    soup = bs(res.text, 'lxml')
    table = soup.find_all('table')[0]

    # Initializing list to store date, open, and close values
    # for GSPC
    dates = []
    opens = []
    closes = []
    
    # Looping through the soup lxml text table format
    # and splitting each row as a individual string
    # and parsing string to retrieve the date,
    # open, and close information.
    i = 1
    end_row = 3
    for row in table.find_all('tr'):
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')
        
        # row_items is parsed string for each current row where each
        # item in list is the date, open, high, low, close, and volume
        row_items = [item.split('>')[1] for item in [string.split('</span>')[0] for string in individual_row[0].split('<span ')[1:]]]
        
        if i == 1:
            # Skip first row because they are column headers
            i += 1
            continue
        elif i == end_row:
            break
        else:
            # Append necessary items to initialized lists for 
            # dataframe storage
            dates.append(row_items[0])
            opens.append(float(row_items[1].replace(',','')))
            closes.append(float(row_items[5].replace(',','')))
        i += 1
    
    # Return dataframe of necessary values
    return pd.DataFrame({ticker + ' Open': opens,ticker + ' Close': closes}, index = dates)

'''
Helper function to pull all relevant current data from yahoo finance using
latest_yahoo function call
'''
def yahoo_latest_data():
    spx_current = latest_yahoo()['SPX Close'][0]
    vix_current = latest_yahoo('VIX')['VIX Close'][0]
    skew_current = latest_yahoo('SKEW')['SKEW Close'][0]
    vvix_current = latest_yahoo('VVIX')['VVIX Close'][0]
    return spx_current, vix_current, skew_current, vvix_current

'''
Function for calculating the single day implied VaR for the SP 500 index
using VIX, SKEW, and the SPX spot.
Inputs:
 - rolling_window [int] - for the number of days to expiry of put option
 - var_pct [float] - for the VaR level
 - option [string of length 1] - for put or call
'''
def implied_spx_var(rolling_window, var_pct, option = 'P'):
    spx, vix, skew, vvix = yahoo_latest_data()
    
    alpha = -(skew - 100)/10
    period_vix = (np.sqrt(((vix*vix)/365)*1.5)/100)*np.sqrt(rolling_window)
    if option == 'C':
        var_pct = 1 - var_pct
        pct_var = norm.ppf(var_pct, 0, period_vix)
    else:
        pct_var = skn.ppf(var_pct, alpha, 0, period_vix)
    strike_suggestion = spx*np.exp(pct_var)#(1 + pct_var)
    # print('VaR return percent for SPX is: ' + str(round(pct_var*100,2)))
    # print('Suggested SPX strike: ' + str(np.floor(spx_k_suggestion)))
    var_spx_return = str(round(pct_var*100,2))
    
    return strike_suggestion, var_spx_return

In [12]:
# Dash browser application

app = dash.Dash()

app.layout = html.Div([
    html.Div([
                html.Label('Days to Expiry:'),
                dcc.Input(id='dte-input-state', type='text', value='1'),
            ],
                className='six columns',
            ),
    html.Div([
                html.Label('VaR Threshold:'),
                dcc.Input(id='var-input-state', type='text', value='0.0005'),
            ],
                className='six columns',
            ),
    html.Button(id='submit-button', n_clicks=0, children='Submit'),
    html.Div(id='output-state')
])


@app.callback(Output('output-state', 'children'),
              [Input('submit-button', 'n_clicks')],
              [State('dte-input-state', 'value'),
               State('var-input-state', 'value')])
def update_output(n_clicks, input1, input2):
    strike_suggestion, var_spx_return = implied_spx_var(float(input1), float(input2), option = 'P')
    
    return u'''
        Submitted {} times \n
        Suggested SPX strike: {} \n
        VaR return percent for SPX is: {}%
    '''.format(n_clicks, strike_suggestion, var_spx_return)


if __name__ == '__main__':
    app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Feb/2018 12:03:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2018 12:03:14] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2018 12:03:14] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2018 12:03:14] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2018 12:03:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2018 12:03:23] "POST /_dash-update-component HTTP/1.1" 200 -
